# Análise de Sentimentos com Lógica Fuzzy

In [1]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 10.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import re
import skfuzzy as fuzz
from skfuzzy import control as ctrl

## Vocabulários utilizados

In [11]:
palavras_positivas = [
    "ótimo", "excelente", "incrível", "fantástico", "maravilhoso", "feliz", "surpreendente", "bom", "gostei",
    "sensacional", "top", "legal", "satisfeito", "adoro", "amei", "perfeito", "eficiente", "agradável", "funcional", "funciona"
]

palavras_negativas = [
    "péssimo", "horrível", "terrível", "desagradável", "decepcionante", "triste", "frustrante", "lento", "ruim",
    "insuportável", "pior", "detestei", "odiei", "inaceitável", "horrendo", "falho", "bugado", "inútil", "desrespeitoso"
]

intensificadores = [
    "muito", "bastante", "extremamente", "incrivelmente", "realmente", "completamente", "totalmente", "super", "mega",
    "absolutamente"
]

negacoes = [
    "não", "jamais", "nenhum", "nem", "nada", "nunca", "de jeito nenhum", "de forma alguma", "zero", "sem", "embora"
]

## Comentários para análise

In [4]:
comentarios_exemplo = [
    "Esse produto é absolutamente incrível, amei cada detalhe!",
    "Não gostei do atendimento. Foi lento e extremamente desagradável.",
    "O serviço foi ótimo, mas o site é completamente bugado.",
    "Realmente fantástico! Super recomendo a todos!",
    "Nada funciona, é o pior serviço que já usei.",
    "Muito satisfeito com a compra, chegou antes do prazo.",
    "Completamente inaceitável. Odiei!",
    "Achei funcional, embora o design não seja legal.",
    "Mega feliz com o resultado, top demais!",
    "Não é ruim, mas esperava mais. Fiquei decepcionado."
]

## Função para extrair frequências

In [5]:
def calcular_frequencias(texto):
    texto = texto.lower()
    palavras = re.findall(r'\b\w+\b', texto)
    total = len(palavras) or 1

    def freq(lista):
        return sum(p in lista for p in palavras) / total

    return {
        "FP": freq(palavras_positivas),
        "FN": freq(palavras_negativas),
        "I": freq(intensificadores),
        "N": freq(negacoes)
    }

## Sistema fuzzy

In [6]:
fp = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'Frequência Positiva')
fn = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'Frequência Negativa')
i = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'Intensificadores')
n = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'Negações')
ps = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'Polaridade Sentimento')

for var in [fp, fn, i, n]:
    var['Baixa'] = fuzz.trimf(var.universe, [0.0, 0.0, 0.3])
    var['Média'] = fuzz.trimf(var.universe, [0.2, 0.4, 0.6])
    var['Alta'] = fuzz.trimf(var.universe, [0.4, 1.0, 1.0])

ps['Negativa'] = fuzz.trimf(ps.universe, [0.0, 0.0, 0.3])
ps['Neutra'] = fuzz.trimf(ps.universe, [0.2, 0.4, 0.6])
ps['Positiva'] = fuzz.trimf(ps.universe, [0.4, 1.0, 1.0])

rules = [
    # Regras POSITIVAS
    ctrl.Rule(fp['Alta'] & fn['Baixa'], ps['Positiva']),
    ctrl.Rule(fp['Alta'] & fn['Média'] & n['Baixa'], ps['Positiva']),
    ctrl.Rule(fp['Média'] & fn['Baixa'] & i['Média'], ps['Positiva']),
    ctrl.Rule(fp['Média'] & fn['Baixa'] & i['Alta'], ps['Positiva']),
    ctrl.Rule(fp['Alta'] & i['Alta'] & fn['Baixa'], ps['Positiva']),

    # Regras NEGATIVAS
    ctrl.Rule(fn['Alta'], ps['Negativa']),
    ctrl.Rule(fn['Média'] & fp['Baixa'], ps['Negativa']),
    ctrl.Rule(fn['Média'] & n['Alta'], ps['Negativa']),
    ctrl.Rule(fn['Média'] & n['Média'] & fp['Baixa'], ps['Negativa']),
    ctrl.Rule(fn['Média'] & i['Alta'] & fp['Baixa'], ps['Negativa']),
    ctrl.Rule(fp['Baixa'] & fn['Baixa'] & n['Média'] & i['Alta'], ps['Negativa']),

    # Regras NEUTRAS
    ctrl.Rule(fp['Média'] & fn['Média'] & n['Baixa'], ps['Neutra']),
    ctrl.Rule(fp['Baixa'] & fn['Baixa'] & i['Baixa'] & n['Baixa'], ps['Neutra']),
    ctrl.Rule(fp['Alta'] & fn['Alta'], ps['Neutra']),
    ctrl.Rule(fp['Média'] & fn['Média'] & n['Alta'], ps['Neutra']),
    ctrl.Rule(fp['Média'] & fn['Baixa'] & n['Alta'], ps['Neutra']),
]


sentiment_ctrl = ctrl.ControlSystem(rules)

## Execução da análise

In [12]:
resultados = []

for comentario in comentarios_exemplo:
    freqs = calcular_frequencias(comentario)
    sentiment_sim = ctrl.ControlSystemSimulation(sentiment_ctrl)

    try:
        sentiment_sim.input['Frequência Positiva'] = freqs["FP"]
        sentiment_sim.input['Frequência Negativa'] = freqs["FN"]
        sentiment_sim.input['Intensificadores'] = freqs["I"]
        sentiment_sim.input['Negações'] = freqs["N"]

        sentiment_sim.compute()
        polaridade = sentiment_sim.output.get('Polaridade Sentimento', 0.5)

        if polaridade < 0.3:
            classificacao = "NEGATIVA"
        elif polaridade < 0.6:
            classificacao = "NEUTRA"
        else:
            classificacao = "POSITIVA"

    except Exception as e:
        print(f"[ERRO] Comentário: '{comentario}'")
        print(f"→ Erro detectado: {e}")
        polaridade = 0.5
        classificacao = "NEUTRA"

    resultados.append({
        "Comentário": comentario,
        "FP": round(freqs["FP"], 2),
        "FN": round(freqs["FN"], 2),
        "I": round(freqs["I"], 2),
        "N": round(freqs["N"], 2),
        "Polaridade": round(polaridade, 3),
        "Classificação": classificacao
    })

df_resultados = pd.DataFrame(resultados)
df_resultados


,Comentário,FP,FN,I,N,Polaridade,Classificação
0,"Esse produto é absolutamente incrível, amei ca...",0.25,0.00,0.12,0.00,0.644,POSITIVA
1,Não gostei do atendimento. Foi lento e extrema...,0.11,0.22,0.11,0.11,0.339,NEUTRA
2,"O serviço foi ótimo, mas o site é completament...",0.10,0.10,0.10,0.00,0.400,NEUTRA
3,Realmente fantástico! Super recomendo a todos!,0.17,0.00,0.33,0.00,0.500,NEUTRA
4,"Nada funciona, é o pior serviço que já usei.",0.11,0.11,0.00,0.11,0.400,NEUTRA
5,"Muito satisfeito com a compra, chegou antes do...",0.11,0.00,0.11,0.00,0.400,NEUTRA
6,Completamente inaceitável. Odiei!,0.00,0.67,0.33,0.00,0.120,NEGATIVA
7,"Achei funcional, embora o design não seja legal.",0.25,0.00,0.00,0.25,0.500,NEUTRA
8,"Mega feliz com o resultado, top demais!",0.29,0.00,0.14,0.00,0.737,POSITIVA
9,"Não é ruim, mas esperava mais. Fiquei decepcio...",0.00,0.12,0.00,0.12,0.400,NEUTRA
